In [92]:
import pandas as pd
import gpxpy.geo
df = pd.read_csv("../data/trips.csv")
df['origin_coordinates'] = 'temp'
df['destination_coordinates'] = 'temp'

In [93]:
from requests import Session
import json

hereId = "uHmohQEsYnwhA4FGz1Jw"
hereCode = "hKz7JnGqtqj5UECMSBqDHw"


session = Session()
def place(search_value):
    
    #weird fix for search returning wrong frankfurt airport
    if search_value == "Frankfurt Airport":
        return [50.05056,8.56442]
    else:
        search_value = search_value.replace(" ", "+")
        base = "https://places.cit.api.here.com/places/v1/autosuggest?at=90.0000,-142.5048&q=" + search_value + "&app_id=" + hereId + "&app_code=" + hereCode
        res = session.get(base)
        data = json.loads(res.content)
        if data['results'] == []:
            coordinates_text = "null"
        else:
            coordinates_text = data['results'][0]['position']
        return coordinates_text 

In [94]:
for x in range(0,df.shape[0]):
    df['origin_coordinates'][x] = place(df['origin'][x])
    df['destination_coordinates'][x] = place(df['destination'][x])

In [95]:
df = df.to_dict('records')


In [96]:
def coord(origin, desti):
    print origin
    print desti
    if ',' in origin and ',' in desti:
        one = [float(origin.split(',')[1]), float(origin.split(',')[0])]
        two = [float(desti.split(',')[1]), float(desti.split(',')[0])]
        return [one, two]
    else:
        return [origin, desti]

In [97]:
#swap lat long
for x in range(0,len(df)):
    df[x]['origin_coordinates'] = [df[x]['origin_coordinates'][1], df[x]['origin_coordinates'][0]]
    df[x]['destination_coordinates'] = [df[x]['destination_coordinates'][1], df[x]['destination_coordinates'][0]]

In [98]:
def distance(origin, destination):
    distance = (gpxpy.geo.haversine_distance(origin[1], origin[0], destination[1], destination[0])) * 0.000621371
    return distance

In [102]:
df

[{'airline': 'Southwest',
  'class': 'Economy',
  'date': '2/24/17',
  'destination': 'San Diego Airport',
  'destination_coordinates': [-117.16171, 32.71568],
  'origin': 'Seattle-Tacoma Airport',
  'origin_coordinates': [-122.44164, 47.25514],
  'type': 'Personal'},
 {'airline': 'Southwest',
  'class': 'Economy',
  'date': '2/26/17',
  'destination': 'Seattle-Tacoma Airport',
  'destination_coordinates': [-122.44164, 47.25514],
  'origin': 'Oakland Airport',
  'origin_coordinates': [-122.27307, 37.80508],
  'type': 'Personal'},
 {'airline': 'Southwest',
  'class': 'Economy',
  'date': '2/26/17',
  'destination': 'Oakland Airport',
  'destination_coordinates': [-122.27307, 37.80508],
  'origin': 'San Diego Airport',
  'origin_coordinates': [-117.16171, 32.71568],
  'type': 'Personal'},
 {'airline': 'United',
  'class': 'Economy',
  'date': '3/19/17',
  'destination': 'Cabo San Lucas Airport',
  'destination_coordinates': [-109.92628, 22.89406],
  'origin': 'San Francisco Airport',
  '

In [103]:
layer = []

for x in range(0,len(df)):
    layer.append({
        "destination": df[x]['destination'],
        "origin": df[x]['origin'],
        "type": df[x]['type'],
        "airline": df[x]['airline'],
        "ports": [df[x]['destination'], df[x]['origin']],
        "date": df[x]['date'],
        "sourcePosition": df[x]['origin_coordinates'],
        "targetPosition": df[x]['destination_coordinates'],
        "distance": distance(df[x]['origin_coordinates'], df[x]['destination_coordinates']),
    })
  
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for x in range(0,len(df)):
    geojson['features'].append({
        "geometry": {
            "type": "LineString",
            "coordinates": [df[x]['origin_coordinates'], df[x]['destination_coordinates']]
        },
        "properties": {
            "destination": df[x]['destination'],
            "origin": df[x]['origin'],
            "type": df[x]['type'],
            "airline": df[x]['airline'],
            "ports": [df[x]['destination'], df[x]['origin']],
            "date": df[x]['date'],
            "sourcePosition": df[x]['origin_coordinates'],
            "targetPosition": df[x]['destination_coordinates'],
            "distance": distance(df[x]['origin_coordinates'], df[x]['destination_coordinates'])
        },
        "type": "Feature",
        "id": x
    })
    


In [104]:
# with open('../data/trips.geojson', 'w') as fp:
#     json.dump(geojson, fp)

In [105]:
import requests
headers = {
    'accept': '*/*',
    'Authorization': 'Bearer {token}',
    'Content-Type': 'application/geo+json',
}

response = requests.put('https://xyz.api.here.com/hub/spaces/79PII7Rm/features', headers=headers, data=json.dumps(geojson['features']))
